In [1]:
"""
PRT-DeepONet_irreversible_sorption (Steady-State, GDF-only, Minimal Runner)
- Trunk input: [x, y, GDF_inlet_norm]
- Uses ONLY two prebuilt datasets:
    Steady_train_dataset.pt
    Steady_test_dataset.pt
- All paths are absolute by default (modify where marked)

Placeholders to change:
    >>> CHANGE HERE: use your own absolute path
"""

'\nPRT-DeepONet_irreversible_sorption (Steady-State, GDF-only, Minimal Runner)\n- Trunk input: [x, y, GDF_inlet_norm]\n- Uses ONLY two prebuilt datasets:\n    Steady_train_dataset.pt\n    Steady_test_dataset.pt\n- All paths are absolute by default (modify where marked)\n\nPlaceholders to change:\n    >>> CHANGE HERE: use your own absolute path\n'

In [2]:
# ===============================
# 0) Imports & deterministic setup
# ===============================
import os
import copy
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [3]:
# ===============================
# 1) Absolute paths (edit here)
# ===============================
# >>> CHANGE HERE: absolute paths to your saved datasets
TRAIN_DATASET_PATH = "/home/yehoon/npz/model/Steady_train_dataset.pt"
TEST_DATASET_PATH  = "/home/yehoon/npz/model/Steady_test_dataset.pt"

# >>> CHANGE HERE: output paths for model and results
MODEL_SAVE_PATH    = "/home/yehoon/npz/model/PRT_Steady_state/PRT-DeepONet_irreversible_sorption.pt"
RESULT_DIR         = "/home/yehoon/npz/model/PRT_Steady_state/PRT_Steady_result"

os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

In [4]:
# ===============================
# 2) Load datasets & sanity check
# ===============================
if not (os.path.exists(TRAIN_DATASET_PATH) and os.path.exists(TEST_DATASET_PATH)):
    raise FileNotFoundError(
        "Dataset files not found.\n"
        f"- {TRAIN_DATASET_PATH}\n- {TEST_DATASET_PATH}"
    )

train_dataset = torch.load(TRAIN_DATASET_PATH, map_location="cpu")
test_dataset  = torch.load(TEST_DATASET_PATH,  map_location="cpu")

def _peek_shapes(ds, name="dataset"):
    b1, b2, trk, tgt = ds.tensors
    assert b1.ndim == 4 and b1.shape[1] == 1, f"{name}: branch1 expected (N,1,H,W)"
    assert b2.ndim == 2 and b2.shape[1] == 2, f"{name}: branch2 expected (N,2)"
    assert trk.ndim == 4 and trk.shape[1] == 1 and trk.shape[-1] == 3, f"{name}: trunk expected (N,1,L,3)"
    assert tgt.ndim == 4 and tgt.shape[-1] == 1, f"{name}: target expected (N,H,W,1)"
    return b1.shape[2], b1.shape[3]

H, W = _peek_shapes(test_dataset, "test_dataset")
print(f"[INFO] Grid size detected: H={H}, W={W}")


[INFO] Grid size detected: H=64, W=148


In [5]:
# ===============================
# 3) Network components
# ===============================
class BranchCNN(nn.Module):
    """Branch network for geometry (CNN)"""
    def __init__(self, in_ch=1, out_dim=128, num_blocks=5, nx=64, ny=148):
        super().__init__()
        chs = [in_ch, 16, 32, 64, 128, 256][:num_blocks+1]
        layers = []
        for i in range(num_blocks):
            layers += [
                nn.Conv2d(chs[i], chs[i+1], kernel_size=3, stride=1, padding=1),
                nn.SiLU(),
                nn.AvgPool2d(2),
            ]
        self.features = nn.Sequential(*layers)
        h, w = nx, ny
        for _ in range(num_blocks):
            h //= 2; w //= 2
        self.fc = nn.Linear(chs[num_blocks]*h*w, out_dim)

    def forward(self, x):
        x = self.features(x)             # (B,C,h,w)
        x = x.view(x.size(0), -1)
        return self.fc(x)                 # (B,out_dim)

class BranchFNN(nn.Module):
    """Branch network for parameters (Pe, DaA)"""
    def __init__(self, in_dim=2, out_dim=128, hidden=128, layers=3):
        super().__init__()
        L = [nn.Linear(in_dim, hidden), nn.SiLU()]
        for _ in range(layers-2):
            L += [nn.Linear(hidden, hidden), nn.SiLU()]
        L += [nn.Linear(hidden, out_dim)]
        self.net = nn.Sequential(*L)
    def forward(self, x):
        return self.net(x)

class Trunk(nn.Module):
    """Trunk network for coordinates (x, y, GDF_inlet_norm)"""
    def __init__(self, in_dim=3, out_dim=128, layers=8, width=128):
        super().__init__()
        L = [nn.Linear(in_dim, width), nn.SiLU()]
        for _ in range(layers-2):
            L += [nn.Linear(width, width), nn.SiLU()]
        L += [nn.Linear(width, out_dim)]
        self.net = nn.Sequential(*L)
    def forward(self, x):
        return self.net(x)

class PRT_DeepONet_irreversible_sorption(nn.Module):
    """y(x) = sum_k B_geom_k * B_param_k * T_k(x) + b"""
    def __init__(self, nx=64, ny=148, out_dim=128):
        super().__init__()
        self.nx, self.ny = nx, ny
        self.branch_geom = BranchCNN(1, out_dim, num_blocks=5, nx=nx, ny=ny)
        self.branch_param = BranchFNN(2, out_dim, hidden=128, layers=3)
        self.trunk = Trunk(in_dim=3, out_dim=out_dim, layers=8, width=128)
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, b1, b2, trunk_in):
        B, _, L, D = trunk_in.shape
        t = self.trunk(trunk_in.view(B*L, D)).view(B, L, -1).unsqueeze(1)   # (B,1,L,C)
        g = self.branch_geom(b1).unsqueeze(1).unsqueeze(2)                  # (B,1,1,C)
        p = self.branch_param(b2).unsqueeze(1).unsqueeze(2)                 # (B,1,1,C)
        y = (g * p * t).sum(-1) + self.bias                                 # (B,1,L)
        return y.view(B, self.nx, self.ny, 1)                               # (B,H,W,1)

In [6]:
# ===============================
# 4) Training & evaluation
# ===============================
def train_model(model, train_dataset, test_dataset, num_epochs=200, lr=1e-3, batch_size=25, patience=15):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.HuberLoss(delta=1.0)
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

    best_loss = float("inf")
    best_state = None
    counter = 0
    train_losses, test_losses = [], []

    for epoch in range(num_epochs):
        model.train()
        tr_loss = 0.0
        for b1, b2, trk, tgt in train_loader:
            b1, b2, trk, tgt = b1.to(device), b2.to(device), trk.to(device), tgt.to(device)
            optimizer.zero_grad(set_to_none=True)
            with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=torch.cuda.is_available()):
                pred = model(b1, b2, trk)
                loss = criterion(pred, tgt)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            tr_loss += loss.item()
        avg_tr = tr_loss / len(train_loader)
        train_losses.append(avg_tr)

        # validation
        model.eval()
        te_loss = 0.0
        with torch.no_grad():
            for b1, b2, trk, tgt in test_loader:
                b1, b2, trk, tgt = b1.to(device), b2.to(device), trk.to(device), tgt.to(device)
                with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=torch.cuda.is_available()):
                    pred = model(b1, b2, trk)
                    loss = criterion(pred, tgt)
                te_loss += loss.item()
        avg_te = te_loss / len(test_loader)
        test_losses.append(avg_te)

        print(f"[{epoch+1:03d}] Train {avg_tr:.6f} | Test {avg_te:.6f}")

        if avg_te < best_loss - 1e-5:
            best_loss = avg_te
            best_state = copy.deepcopy(model.state_dict())
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping.")
                break

    if best_state is not None:
        model.load_state_dict(best_state)
    torch.save(model.state_dict(), MODEL_SAVE_PATH)
    print(f"[SAVE] Model saved to {MODEL_SAVE_PATH}")

    # loss curve
    plt.plot(train_losses, label="Train")
    plt.plot(test_losses, label="Test")
    plt.xlabel("Epoch"); plt.ylabel("Loss")
    plt.legend(); plt.tight_layout()
    plt.savefig(os.path.join(RESULT_DIR, "loss_curve.png"), dpi=150)
    plt.close()


In [7]:
# ===============================
# 5) Run training
# ===============================
if __name__ == "__main__":
    model = PRT_DeepONet_irreversible_sorption(nx=H, ny=W, out_dim=128)
    train_model(model, train_dataset, test_dataset,
                num_epochs=200, lr=1e-3, batch_size=25, patience=15)
    print("[DONE]")

[001] Train 0.004099 | Test 0.001239
[002] Train 0.001160 | Test 0.001002
[003] Train 0.001007 | Test 0.000855
[004] Train 0.000894 | Test 0.000848
[005] Train 0.000856 | Test 0.000840
[006] Train 0.000821 | Test 0.000839
[007] Train 0.000783 | Test 0.000753
[008] Train 0.000772 | Test 0.000726
[009] Train 0.000719 | Test 0.000787
[010] Train 0.000719 | Test 0.000709
[011] Train 0.000664 | Test 0.000714
[012] Train 0.000585 | Test 0.000606
[013] Train 0.000536 | Test 0.000596
[014] Train 0.000487 | Test 0.000559
[015] Train 0.000455 | Test 0.000574
[016] Train 0.000410 | Test 0.000581
[017] Train 0.000361 | Test 0.000560
[018] Train 0.000326 | Test 0.000551
[019] Train 0.000293 | Test 0.000537
[020] Train 0.000270 | Test 0.000569
[021] Train 0.000246 | Test 0.000565
[022] Train 0.000228 | Test 0.000567
[023] Train 0.000219 | Test 0.000575
[024] Train 0.000204 | Test 0.000540
[025] Train 0.000192 | Test 0.000567
[026] Train 0.000181 | Test 0.000537
[027] Train 0.000175 | Test 0.000534
[